# Workshop: Generative Adversarial Networks

In this workshop, we will implement a Generative Adversarial Network (GAN) to generate images of handwritten digits. We will use the MNIST dataset, which contains 70,000 images of handwritten digits. Each image is a 28x28 grayscale image, and each pixel has a value between 0 and 255.

The GAN will consist of two neural networks: a generator and a discriminator. The generator will take a random vector as input and output an image. The discriminator will take an image as input and output a probability that the image is real (as opposed to generated). The two networks will be trained together, with the goal of the generator producing images that are indistinguishable from real images.

In [11]:
IMG_ROWS = 28
IMG_COLS = 28
CHANNELS = 1

IMG_SHAPE = (IMG_ROWS, IMG_COLS, CHANNELS)

LATENT_DIM = 100

#### Exercise 1: Preparing the data

In order to train the GAN, we need to prepare the data. We will use the MNIST dataset, which is available in the `keras.datasets` module. The dataset is split into a training set and a test set, but we will only use the X from the training set.

We will create a function that will load the dataset, normalize the pixel values to be between -1 and 1, and also reshape the images to be 28x28x1 instead of 28x28.

In [12]:
from keras.datasets import mnist

import numpy as np

def prepare_data():
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1

    # Normalize data -> from 0 - 127 to -1 - 1
    X_train = X_train / 127.5 - 1.
    # Add channel dimension
    X_train = np.expand_dims(X_train, axis=3)

    return X_train
    

#### Exercise 2: Creating the generator

The generator will take a random vector (called: noise) as input and output an image, in order to ensure that the generator is able to produce different images.

For the generator, we will use a neural network with 3 layers.
- The first layer will be a dense layer with 256 neurons, a Leaky ReLU activation function with an alpha of 0.2 and a batch normalization layer with a momentum of 0.8.
- The second layer will be a dense layer with 512 neurons, a Leaky ReLU activation function with an alpha of 0.2 and a batch normalization layer with a momentum of 0.8.
- The third layer will be a dense layer with 1024 neurons, a Leaky ReLU activation function with an alpha of 0.2 and a batch normalization layer with a momentum of 0.8.
- The fourth layer will be a dense layer with the number of neurons equal to the number of pixels in the image (28x28), and a tanh activation function.

After the fourth layer, we will reshape the output to be 28x28x1 (hint: use the Reshape layer).

In [13]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model

def build_generator():

    model = Sequential()

    model.add(Dense(256, input_dim=LATENT_DIM))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(IMG_SHAPE), activation='tanh'))
    model.add(Reshape(IMG_SHAPE))

    model.summary()

    noise = Input(shape=(LATENT_DIM,))
    img = model(noise)

    return Model(noise, img)

#### Exercise 3: Creating the discriminator

The discriminator will take an image as input and output a probability that the image is real (as opposed to generated).

For the discriminator, we will use a neural network with 3 layers.
- The first layer will be a dense layer with 512 neurons, a Leaky ReLU activation function with an alpha of 0.2.
- The second layer will be a dense layer with 256 neurons, a Leaky ReLU activation function with an alpha of 0.2.
- The third layer will be a dense layer with 1 neuron, and a sigmoid activation function.

In [14]:
def build_discriminator():

    model = Sequential()

    model.add(Flatten(input_shape=IMG_SHAPE))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=IMG_SHAPE)
    validity = model(img)

    return Model(img, validity)

#### Exercise 4: Displaying the images

In order to see the images that the generator produces, we will create a function that will save a grid of images to a file.

In [15]:
import matplotlib.pyplot as plt

def sample_images(epoch, generator):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, LATENT_DIM))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

#### Exercise 5: Creating the GAN

Now that we have the generator and the discriminator, we can create the GAN. The GAN will consist of the generator and the discriminator, connected in a sequential manner. The discriminator will be frozen during training, so that only the generator will be trained.

In order to train the GAN, we will use the Adam optimizer with a learning rate of 0.0002 and a momentum of 0.5. We will use binary crossentropy as the loss function.

In [16]:
from keras.optimizers.legacy import Adam

OPTIMIZER = Adam(0.0002, 0.5)

def create_model():

    # Create the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy',
        optimizer=OPTIMIZER,
        metrics=['accuracy'])
    
    # Create the generator
    generator = build_generator()

    # The generator takes noise as input and generates imgs
    z = Input(shape=(LATENT_DIM,))
    img = generator(z)

    # For the combined model we will only train the generator
    discriminator.trainable = False
    # The discriminator takes generated images as input and determines validity
    validity = discriminator(img)
    
    combined = Model(z, validity)
    # The combined model  (stacked generator and discriminator)
    # Trains the generator to fool the discriminator
    combined.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)
    
    return discriminator, generator, combined
    


#### Exercise 6: Training the GAN

Now that we have the GAN, we need to create a function that will train the GAN.

The training process will consist of multiple steps:
1. we will select a random batch of images from the training set.
2. we will generate a random batch of images using the generator.
3. we will train the discriminator on the real images
4. we will train the discriminator on the generated images
5. We will calculate the loss for the discriminator
6. We will train the Combined model (the GAN) with noise as input and ones as output
7. We will save images to a file every 200 epochs


> **Note:** The training process of a GAN is very long, so you will need a lot of epochs in order to get good results. We recommend 2000 epochs or more.

In [17]:

class GAN():
    def __init__(self):
        self.discriminator, self.generator, self.combined  = create_model()

    def train(self, epochs, batch_size=128, sample_interval=50):

        X_train = prepare_data()

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid) # Learn to recognize real images
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake) # Learn to recognize fake images
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # Average the two losses

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                sample_images(epoch, self.generator)

#### Exercise 7: Generating images

Now that we have trained the GAN, we can use the generator to generate images.

We will create a function that will generate a random batch of images using the generator, and then save the images to a file.

In [18]:
gan = GAN()
gan.train(epochs=2001, batch_size=32, sample_interval=200)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533505 (2.04 MB)
Trainable params: 53350

# Conclusion

In this workshop, we have implemented a Generative Adversarial Network (GAN) to generate images of handwritten digits. We have used the MNIST dataset, which contains 70,000 images of handwritten digits. Each image is a 28x28 grayscale image, and each pixel has a value between 0 and 255.

# To go further

To go further, now that you know how to implement a GAN, you can try to implement a GAN to generate images of faces. You can use the CelebA dataset, which contains 202,599 images of faces. Each image is a 178x218 RGB image, and each pixel has a value between 0 and 255. Otherwise, you can try to implement diffent types of GANs, such as a Conditional GAN (CGAN) or a Deep Convolutional GAN (DCGAN).